# GENIE Analysis
This analysis highlights useful statistics for fusion data from the AACR Project Genie cohort

The cells below are run to set environment variables and load in FUSOR.

In [1]:
from os import environ
import logging
from pathlib import Path

# These are the configurations for the UTA and SeqRepo databases. These should
# be adjusted by the user based on the locations where these databases exist.
environ["UTA_DB_URL"] = "postgresql://anonymous@localhost:5432/uta/uta_20241220"
environ["SEQREPO_ROOT_DIR"] = "/usr/local/share/seqrepo/2024-12-20"

# Cool-Seq-Tool will log warning messages for reasons including having breakpoints
# that occur outside of the transcript boundaries and having gene partners 
# that do not have MANE select transcripts. We are silencing these warning 
# messages in the output, as FUSOR objects can still be generated for these
# cases.
logging.getLogger("cool_seq_tool").setLevel(logging.ERROR)

In [2]:
from fusor.fusor import FUSOR
from fusor.fusion_matching import FusionMatcher

fusor = FUSOR()

***Using Gene Database Endpoint: http://localhost:8000***


## 1. Filtering Operations
The cells below are used to perform filtering to extract fusion events from the GENIE structural variant data

In [3]:
import pandas as pd
import numpy as np

genie_data = pd.read_csv("data/data_sv.txt", sep="\t", dtype=str)
f"The number of rows in the GENIE structural variant file is {len(genie_data)}"

'The number of rows in the GENIE structural variant file is 54124'

We want to filter for rows where both `Site1_Hugo_Symbol` and `Site2_Hugo_Symbol` are non-Null values and are different from each other. These conditions help to filter on potential fusion events in the file.

In [4]:
genie_fusions = genie_data[
    genie_data["Site1_Hugo_Symbol"].notna() &
    genie_data["Site2_Hugo_Symbol"].notna() &
    (genie_data["Site1_Hugo_Symbol"] != genie_data["Site2_Hugo_Symbol"])
]
f"The number of potential fusion events is: {len(genie_fusions)}"

'The number of potential fusion events is: 32092'

There are some rows where `INTERGENIC` and `INTRAGENIC` are used as a gene symbol. These do not indicate fusion events, so we want to filter on these as well.

In [5]:
genie_fusions = genie_fusions[
    (~genie_fusions["Site1_Hugo_Symbol"].isin(["INTERGENIC", "INTRAGENIC"])) &
    (~genie_fusions["Site2_Hugo_Symbol"].isin(["INTERGENIC", "INTRAGENIC"]))
]
f"The number of potential fusion events after removing for INTERGENIC and INTRAGENIC entries is: {len(genie_fusions)}"

'The number of potential fusion events after removing for INTERGENIC and INTRAGENIC entries is: 21894'

## 2. MSK
The next cells run analyses to highlight challenges with standardizing fusion data from the MSK center.

The cell below defines helper functions that will be used

In [6]:
def is_ordered_correctly(five_prime_gene: str, three_prime_gene, annot: str) -> bool:
    """Determine if the fusion partners are oriented correctly for the MSK subset
    
    :param five_prime_gene: The assumed 5' gene partner, provided in the Site1_Hugo_Symbol column
    :param three_prime_gene: The assumed 3' gene partner, provided in the Site2_Hugo_Symbol column
    :param annot: The fusion annotation
    :return: ``True`` if the orientation is consistent, ``False`` if not
    """
    elements = annot.split(" ")
    five_prime_index = next((i for i, word in enumerate(elements) if word == five_prime_gene), None)
    three_prime_index = next((i for i, word in enumerate(elements) if word == three_prime_gene), None)
    return bool(five_prime_index < three_prime_index)

def both_genes_present(five_prime_gene: str, three_prime_gene, annot: str) -> bool:
    """Determine if both genes partners are present in the annotation
    
    :param five_prime_gene: The assumed 5' gene partner, provided in the Site1_Hugo_Symbol column
    :param three_prime_gene: The assumed 3' gene partner, provided in the Site2_Hugo_Symbol column
    :param annot: The fusion annotation
    :return: ``True`` if both partners are present, ``False`` if not
    """
    if pd.isna(annot):
        # Return False if the annotation string is not available
        return False
    elements = annot.split(" ")
    if five_prime_gene in elements and three_prime_gene in elements:
        return True
    return False

def are_breakpoints_correct(five_prime_gene: str, three_prime_gene, 
                            five_prime_description: str, three_prime_description) -> bool:
    """Determine if the supplied breakpoints describe the correct partner
    
    :param five_prime_gene: The assumed 5' gene partner, provided in the Site1_Hugo_Symbol column
    :param three_prime_gene: The assumed 3' gene partner, provided in the Site2_Hugo_Symbol column
    :param five_prime_description: The 5' gene partner location description, provided in the Site1_Description column
    :param three_prime_description: The 3' gene partner location description, provided in the Site2_Description column
    """
    if pd.isna(five_prime_description) and pd.isna(three_prime_description):
        # Return True. This corresponds to gene fusion events that were detected by Archer
        return True
    return bool(five_prime_gene in five_prime_description and three_prime_gene in three_prime_description)

In [7]:
msk_data = genie_fusions[genie_fusions["Center"] == "MSK"]
f"The number of filtered fusions from MSK is: {len(msk_data)}"

'The number of filtered fusions from MSK is: 9315'

Select rows where both gene partners appear in the annotation string

In [8]:
msk_data = msk_data[
    msk_data.apply(
        lambda row: both_genes_present(
            row["Site1_Hugo_Symbol"],
            row["Site2_Hugo_Symbol"],
            row["Annotation"]
        ),
        axis=1
    )
]
f"The number of filtered fusions from MSK is: {len(msk_data)}"

'The number of filtered fusions from MSK is: 8348'

There are some rows where the 5' and 3' partners appear to be out of order when examining the annotation string. We want to filter to include rows where we believe the partners to be in order.

In [9]:
msk_data = msk_data[
    msk_data.apply(
        lambda row: is_ordered_correctly(
            row["Site1_Hugo_Symbol"],
            row["Site2_Hugo_Symbol"],
            row["Annotation"]
        ),
        axis=1
    )
]
f"The number of filtered fusions from MSK is: {len(msk_data)}"

'The number of filtered fusions from MSK is: 7268'

There are some rows where the provided genomic breakpoints appear out of order, describing the wrong partner. We want to filter to include rows where we believe the correct genomic breakpoints are provided.

In [10]:
msk_data = msk_data[
    msk_data.apply(
        lambda row: are_breakpoints_correct(
            row["Site1_Hugo_Symbol"],
            row["Site2_Hugo_Symbol"],
            row["Site1_Description"],
            row["Site2_Description"],
        ),
        axis=1
    )
]
f"The number of filtered fusions from MSK is: {len(msk_data)}"

'The number of filtered fusions from MSK is: 4258'

In [11]:
msk_data.to_csv("data/genie_data/msk_data.csv", sep="\t", index=False)

In [12]:
msk_data[msk_data["Site1_Position"].isna()].to_csv("data/genie_data/msk_data_test.csv", sep="\t", index=False)

### Standardize using GENIE translator
The cell below attempts to standardize the 4,258 filtered fusions from MSK

In [13]:
from fusor.harvester import GenieHarvester
from cool_seq_tool.schemas import Assembly

harvester = GenieHarvester(fusor=fusor, assembly=Assembly.GRCH37)
msk_fusions = await harvester.load_records(fusion_path=Path("data/genie_data/msk_data.csv"))

## 3. UHN
The next cells standardizes data from the UHN center. This subset has consistent strcture, so no additional filtering is required

In [18]:
uhn_data = genie_fusions[genie_fusions["Center"] == "UHN"]
f"The number of filtered fusions from MSK is: {len(uhn_data)}"

'The number of filtered fusions from MSK is: 92'

In [19]:
uhn_data.to_csv("data/genie_data/uhn_data.csv", sep="\t", index=False)

### Standardize using GENIE translator
The cell below attempts to standardize the 92 filtered fusions from UHN

In [20]:
uhn_fusions = await harvester.load_records(fusion_path=Path("data/genie_data/uhn_data.csv"))

## 4. VICC
The next cells standardizes data from the VICC center

In [31]:
vicc_data = genie_fusions[genie_fusions["Center"] == "VICC"]
f"The number of filtered fusions from VICC is: {len(vicc_data)}"

'The number of filtered fusions from VICC is: 450'

The VICC data includes rearrangements, which appear to be distinct from fusions. We want to only include fusions, so we will filter on the `Event_Info` field

In [32]:
vicc_data = vicc_data[~vicc_data["Event_Info"].str.contains("truncation|intragenic|rearrangement", case=False, na=False)]
f"The number of filtered fusions from VICC is: {len(vicc_data)}"

'The number of filtered fusions from VICC is: 226'

In [38]:
vicc_data.to_csv("data/genie_data/vicc_data.csv", sep="\t", index=False)

### Standardize using GENIE translator
The cell below attempts to standardize the 226 filtered fusions from VICC

In [39]:
vicc_fusions = await harvester.load_records(fusion_path=Path("data/genie_data/vicc_data.csv"))

## 5. UCSF
The next cells standardizes data from the UCSF center

In [44]:
ucsf_data = genie_fusions[genie_fusions["Center"] == "UCSF"]
f"The number of filtered fusions from VICC is: {len(ucsf_data)}"

'The number of filtered fusions from VICC is: 1466'

Filter to only include fusions. Some non-fusion events, such as rearrangements, may not have passed the initial filters.

In [45]:
ucsf_data = ucsf_data[ucsf_data["Event_Info"].str.contains("fusion", case=False, na=False)]
f"The number of filtered fusions from VICC is: {len(ucsf_data)}"

'The number of filtered fusions from VICC is: 1419'

In [46]:
ucsf_data.to_csv("data/genie_data/ucsf_data.csv", sep="\t", index=False)

### Standardize using GENIE translator
The cell below attempts to standardize the 1,419 filtered fusions from UCSF

In [47]:
ucsf_fusions = await harvester.load_records(fusion_path=Path("data/genie_data/ucsf_data.csv"))

## 6. CHOP
The following cells attempt to standardize data from the CHOP center

In [50]:
chop_data = genie_fusions[genie_fusions["Center"] == "CHOP"]
f"The number of filtered fusions from CHOP is: {len(chop_data)}"

'The number of filtered fusions from CHOP is: 454'

All data from CHOP describe gene fusion events, so no additional filtering is needed. We can move straight to standardization with FUSOR